# Data Cleaning.

In [1]:
import pandas as pd

from src.cleaning_functions import *

from src.text_functions import *

**ÍNDICE:**
1. Nike data.
    - Exploración inicial de los datos.
    - Hashtags.
    - Preparing Data.
    - Subsets.



2. Adidas data.
    - Exploración inicial de los datos.
    - Hashtags.
    - Preparing Data.
    - Subsets.



3. Dataset Completo.


## Nike data.

In [2]:
nike = pd.read_csv('data/nike.csv')

In [3]:
nike_tw = pd.DataFrame(nike.text)

### Exploración inicial de los datos:

In [4]:
nike_tw.shape

(29804, 1)

In [5]:
nike_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29804 entries, 0 to 29803
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    29804 non-null  object
dtypes: object(1)
memory usage: 233.0+ KB


In [6]:
nike_tw.describe()

,text
count,29804
unique,29276
top,@whatudohazz nike
freq,59


In [7]:
nike_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [8]:
nike_tw.duplicated().sum() # Encontramos 528 valores duplicados.

528

In [9]:
drop_duplicates(nike_tw) # Eliminamos los valores duplicados.

In [10]:
nike_tw.shape

(29276, 1)

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DatFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

*Posibles aplicaciones de este análisis:*
- *Análisis del sentimiento de la conversación en torno a un hashtag específico (nos permite analizar la percepción acerca de una campaña online, un evento en directo o similares).*


- *Ranking de hashtags con mayor sentimiento positivo o negativo.*


- *Análisis de utilización de hashtags: podemos analizar elementos más básicos como los hashtags más frecuentes o incluso cruzarlo con otros datos extraídos (datetime de publicación, usuarios que publican el tweet...) para llegar a conclusiones relevantes.*

In [11]:
nike_tw['hashtags'] = nike_tw.text.apply(extract_hashtags)

In [12]:
nike_tw.head()

,text,hashtags
0,@ZubyMusic @LPMisesCaucus Nobody cares about N...,[]
1,@PoojaPraharaj @IndiainUkraine @PMOIndia @MEAI...,[]
2,@DocumentWomen Nike Okundaye. Also knowns Nike...,"[#womengiant, #Documentwomen]"
3,Day 4 of #maxmadness #AirMaxMonth \nNike Air M...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd..."
4,@MagMr44 @soleguru @nikestore @Nike @SneakerAd...,[]


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [13]:
nike_tw.head()

,text,hashtags
0,@ZubyMusic @LPMisesCaucus Nobody cares about N...,[]
1,@PoojaPraharaj @IndiainUkraine @PMOIndia @MEAI...,[]
2,@DocumentWomen Nike Okundaye. Also knowns Nike...,"[#womengiant, #Documentwomen]"
3,Day 4 of #maxmadness #AirMaxMonth \nNike Air M...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd..."
4,@MagMr44 @soleguru @nikestore @Nike @SneakerAd...,[]


In [14]:
nike_tw['text'] = nike_tw.text.apply(basic_cleaning)

In [15]:
nike_tw.head()

,text,hashtags
0,nobody cares about nike in russia russia is al...,[]
1,ye green nike hoodie waala two weeks pehle put...,[]
2,nike okundaye also knowns nike twins seven sev...,"[#womengiant, #Documentwomen]"
3,day four of maxmadness air max month nike air ...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd..."
4,thank you sir,[]


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Esto es lo que haremos en esta etapa del proceso, en este caso sobre los tweets que mencionan a Nike.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [16]:
nike_tw['brand_attribute'] = nike_tw['text'].apply(data_groups)

In [17]:
nike_tw.head()

,text,hashtags,brand_attribute
0,nobody cares about nike in russia russia is al...,[],quality
1,ye green nike hoodie waala two weeks pehle put...,[],quality
2,nike okundaye also knowns nike twins seven sev...,"[#womengiant, #Documentwomen]",quality
3,day four of maxmadness air max month nike air ...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd...",quality
4,thank you sir,[],quality


In [18]:
nike_tw.brand_attribute.value_counts()

quality    23892
price       5384
Name: brand_attribute, dtype: int64

2. Exportamos los subsets de Nike (uno de calidad y otro de precio).

    2.1. Precio:

In [19]:
nike_price = pd.DataFrame(nike_tw.where(nike_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [20]:
nike_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [21]:
nike_price.shape

(5384, 2)

In [22]:
# Exportamos el subset de precio.

nike_price.to_csv('data/nike_price.csv', index = False)

    2.2. Calidad:

In [23]:
nike_quality = pd.DataFrame(nike_tw.where(nike_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [24]:
nike_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [25]:
nike_quality.shape

(23892, 2)

In [26]:
# Exportamos el subset de calidad.

nike_quality.to_csv('data/nike_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Adidas data.

In [27]:
adidas = pd.read_csv('data/adidas.csv')

In [28]:
adidas_tw = pd.DataFrame(adidas.text)

### Exploración inicial de los datos:

In [29]:
adidas_tw.shape

(29816, 1)

In [30]:
adidas_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29816 entries, 0 to 29815
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    29816 non-null  object
dtypes: object(1)
memory usage: 233.1+ KB


In [31]:
adidas_tw.describe()

,text
count,29816
unique,29280
top,@whatudohazz nike
freq,60


In [32]:
adidas_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [33]:
adidas_tw.duplicated().sum() # Encontramos 528 valores duplicados.

536

In [34]:
drop_duplicates(adidas_tw) # Eliminamos los valores duplicados.

In [35]:
adidas_tw.shape

(29280, 1)

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DatFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

In [36]:
adidas_tw['hashtags'] = adidas_tw.text.apply(extract_hashtags)

In [37]:
adidas_tw.head()

,text,hashtags
0,@Coop8517 Because my lad is a div he won't wea...,[]
1,"@shirtsinthshire beside wrong nameset, its lik...",[]
2,@ChefLerrie @Jakjakph1 Yes dude sa Adidas webs...,[]
3,Footlocker is solely focusing on only selling ...,[]
4,adidas Yeezy Boost 350 Pirate Black V1 (Infant...,[]


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [38]:
adidas_tw.head()

,text,hashtags
0,@Coop8517 Because my lad is a div he won't wea...,[]
1,"@shirtsinthshire beside wrong nameset, its lik...",[]
2,@ChefLerrie @Jakjakph1 Yes dude sa Adidas webs...,[]
3,Footlocker is solely focusing on only selling ...,[]
4,adidas Yeezy Boost 350 Pirate Black V1 (Infant...,[]


In [39]:
adidas_tw['text'] = adidas_tw.text.apply(basic_cleaning)

In [40]:
adidas_tw.head()

,text,hashtags
0,because my lad is a div he won t wear adidas o...,[]
1,beside wrong nameset its like fake shirts spac...,[]
2,yes dude sa adidas website mismo yang link sa ...,[]
3,footlocker is solely focusing on only selling ...,[]
4,adidas yeezy boost three five zero pirate blac...,[]


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Ahora llevaremos a cabo este proceso sobre los tweets que mencionan a Adidas.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [41]:
adidas_tw['brand_attribute'] = adidas_tw['text'].apply(data_groups)

In [42]:
adidas_tw.head()

,text,hashtags,brand_attribute
0,because my lad is a div he won t wear adidas o...,[],quality
1,beside wrong nameset its like fake shirts spac...,[],quality
2,yes dude sa adidas website mismo yang link sa ...,[],quality
3,footlocker is solely focusing on only selling ...,[],quality
4,adidas yeezy boost three five zero pirate blac...,[],price


In [43]:
adidas_tw.brand_attribute.value_counts()

quality    23892
price       5388
Name: brand_attribute, dtype: int64

2. Generamos los subsets de Adidas (uno de calidad y otro de precio).

    2.1. Precio:

In [44]:
adidas_price = pd.DataFrame(adidas_tw.where(adidas_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [45]:
adidas_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [46]:
adidas_price.shape

(5388, 2)

In [47]:
# Exportamos el subset de precio.

adidas_price.to_csv('data/adidas_price.csv', index = False)

    2.2. Calidad:

In [48]:
adidas_quality = pd.DataFrame(adidas_tw.where(adidas_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [49]:
adidas_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [50]:
adidas_quality.shape

(23892, 2)

In [51]:
# Exportamos el subset de calidad.

adidas_quality.to_csv('data/adidas_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Dataset Completo.

Primero generamos una nueva columna en la que se recoja la marca a la que hace referencia el tweet (todos los valores de la columna serán 'nike' para el DataFrame de Nike, 'adidas' para el DataFrame de Adidas...).

#### Columna 'brand' para los datos de Nike.

In [52]:
nike_tw.head()

,text,hashtags,brand_attribute
0,nobody cares about nike in russia russia is al...,[],quality
1,ye green nike hoodie waala two weeks pehle put...,[],quality
2,nike okundaye also knowns nike twins seven sev...,"[#womengiant, #Documentwomen]",quality
3,day four of maxmadness air max month nike air ...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd...",quality
4,thank you sir,[],quality


In [53]:
nike_tw = nike_tw.assign(brand='nike')

In [54]:
nike_tw.head()

,text,hashtags,brand_attribute,brand
0,nobody cares about nike in russia russia is al...,[],quality,nike
1,ye green nike hoodie waala two weeks pehle put...,[],quality,nike
2,nike okundaye also knowns nike twins seven sev...,"[#womengiant, #Documentwomen]",quality,nike
3,day four of maxmadness air max month nike air ...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd...",quality,nike
4,thank you sir,[],quality,nike


#### Columna 'brand' para los datos de Adidas.

In [55]:
adidas_tw.head()

,text,hashtags,brand_attribute
0,because my lad is a div he won t wear adidas o...,[],quality
1,beside wrong nameset its like fake shirts spac...,[],quality
2,yes dude sa adidas website mismo yang link sa ...,[],quality
3,footlocker is solely focusing on only selling ...,[],quality
4,adidas yeezy boost three five zero pirate blac...,[],price


In [56]:
adidas_tw = adidas_tw.assign(brand='adidas')

In [57]:
adidas_tw.head()

,text,hashtags,brand_attribute,brand
0,because my lad is a div he won t wear adidas o...,[],quality,adidas
1,beside wrong nameset its like fake shirts spac...,[],quality,adidas
2,yes dude sa adidas website mismo yang link sa ...,[],quality,adidas
3,footlocker is solely focusing on only selling ...,[],quality,adidas
4,adidas yeezy boost three five zero pirate blac...,[],price,adidas


#### Generamos dataset completo.

In [58]:
brands_tw = pd.concat([nike_tw, adidas_tw])

In [59]:
brands_tw.brand.value_counts()

adidas    29280
nike      29276
Name: brand, dtype: int64

#### Exportamos el dataset completo.

In [60]:
brands_tw.to_csv('data/brands.csv', index = False)